In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf


In [2]:
# Đọc dữ liệu
df = pd.read_csv('STORM_RAW_1.csv')

In [3]:
df.head()

,Unnamed: 0,ID,EventName,Year,Month,DisasterType,DisasterSubtype,MainLandfallLocation,Flood,Slide,OFDAResponse,Appeal,Declaration,LandfallMagnitude(kph),LandfallPressure(mb),TotalDeaths,NoInjured,TotalDamage(000US$),TotalDamageAdjusted(000US$),CPI
0,0,STORM_NAN_1953,NaN,1953,9.0,Storm,Tropical cyclone,Mekong River Delta,0.0,0.0,0.0,0.0,0.0,93.0,985.0,1000.0,146.0,89730.0,139143.00,9.156133
1,1,STORM_VIOLET_1964,Violet,1964,9.0,Storm,Tropical cyclone,North Central Coast,NaN,NaN,0.0,0.0,0.0,93.0,985.0,124.0,146.0,10000.0,94354.08,10.598376
2,2,STORM_IRIS_1964,Iris,1964,11.0,Storm,Tropical cyclone,South Central Coast,NaN,NaN,0.0,0.0,0.0,150.0,960.0,5100.0,146.0,70000.0,660478.55,10.598376
3,3,STORM_JOAN_1964,Joan,1964,11.0,Storm,Tropical cyclone,South Central Coast,NaN,NaN,0.0,0.0,0.0,130.0,980.0,2500.0,146.0,15000.0,141531.12,10.598376
4,4,STORM_KATE_1964,Kate,1964,11.0,Storm,Tropical cyclone,Central Highlands,NaN,NaN,0.0,0.0,0.0,130.0,970.0,0.0,146.0,3000.0,28306.22,10.598376


In [4]:
# Xóa các cột không cần thiết
df = df.drop(['OFDAResponse', 'Appeal', 'Declaration', 'CPI'], axis=1)

# Xử lý giá trị NaN
df['Flood'] = df['Flood'].fillna(0)
df['Slide'] = df['Slide'].fillna(0)

In [5]:
unique_values = df['MainLandfallLocation'].unique()

print(unique_values)

['Mekong River Delta' 'North Central Coast' 'South Central Coast'
 'Central Highlands' 'SouthEast' 'Red River Delta' 'NorthEast' nan
 'NorthWest']


In [6]:
# Mapping cho MainLandfallLocation
landfall_mapping = {
    'NorthEast': 1,
    'NorthWest': 2,
    'Red River Delta': 3,
    'North Central Coast': 4,
    'South Central Coast': 5,
    'Central Highlands': 6,
    'SouthEast': 7,
    'Mekong River Delta': 8
}
df['MainLandfallLocation'] = df['MainLandfallLocation'].map(landfall_mapping)

# One-hot encoding cho các cột phân loại
categorical_cols = ['DisasterType', 'DisasterSubtype']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


In [7]:
# Chuẩn hóa các cột số
scaler = StandardScaler()
df[['LandfallMagnitude(kph)', 'LandfallPressure(mb)']] = scaler.fit_transform(df[['LandfallMagnitude(kph)', 'LandfallPressure(mb)']])

# Kiểm tra giá trị NaN
if df.isnull().values.any():
    print("DataFrame contains NaN values. Please check your data.")

# Tạo X và y
predictors = ['LandfallMagnitude(kph)', 'LandfallPressure(mb)', 'Flood', 'Slide', 'Year', 'Month'] + \
             [col for col in df.columns if col.startswith('MainLandfallLocation_') or col.startswith('DisasterSubtype_')]

X = df[predictors]
y = df[['TotalDeaths', 'NoInjured', 'TotalDamageAdjusted(000US$)']]

# Kiểm tra NaN trong y
y = y.fillna(0)  # Hoặc thay thế bằng giá trị trung bình

# Chuẩn hóa dữ liệu X
scaler = StandardScaler()
X = scaler.fit_transform(X)

DataFrame contains NaN values. Please check your data.


In [85]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

**thử với MLP nhưng bị nan**

In [86]:
# Khởi tạo mô hình Sequential
model = Sequential()

# Thêm các hidden layers
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Layer đầu ra (3 node cho 3 feature trong y)
model.add(Dense(3))

# Compile mô hình
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: nan - val_loss: nan
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan - val_loss: nan
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: nan
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: nan
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: nan
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: nan
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: nan - val_loss: nan
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: nan - val_loss: nan
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan - val_loss: nan
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: nan - val_loss: nan
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: nan - val_loss: nan
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: nan
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: nan


In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Example assuming you have already loaded your dataset X (predictors) and y (targets)

# 1. Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Normalize the data (important for dense networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# 3. Build the neural network model
def build_model(input_dim, output_dim):
    model = Sequential()
    
    # Input Layer
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))  # To prevent overfitting
    
    # Hidden Layer 1
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    # Hidden Layer 2
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    # Output Layer - Regression (use 'linear' activation) or Classification (use 'softmax' for multi-class or 'sigmoid' for binary classification)
    model.add(Dense(output_dim, activation='linear'))  # Change activation based on your task
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])  # For regression, you can change this to 'binary_crossentropy' or 'categorical_crossentropy' for classification tasks.
    
    return model

# 4. Define input/output dimensions
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]  # If multi-target regression, otherwise use 1

# 5. Build and compile the model
model = build_model(input_dim, output_dim)

# 6. Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

# 7. Evaluate the model
loss, mae = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}, Validation MAE: {mae}")

c:\Users\anhmi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 16902643712.0000 - mae: 44236.6172 - val_loss: 28470239232.0000 - val_mae: 47764.3945
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 22583275520.0000 - mae: 51556.1172 - val_loss: 28470235136.0000 - val_mae: 47764.3750
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 16426182656.0000 - mae: 44006.8555 - val_loss: 28470231040.0000 - val_mae: 47764.3594
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 21577543680.0000 - mae: 49263.0508 - val_loss: 28470231040.0000 - val_mae: 47764.3477
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 18459719680.0000 - mae: 46809.3242 - val_loss: 28470222848.0000 - val_mae: 47764.3320
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 16573284352.0000 - mae: 42204.0156 - val_loss: 28470220800.0000 - val_mae: 47764.3125
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 16201964544.0000 - mae: 44153.9922 - val_loss: 28470216704.0000 - val_mae: 477